In [93]:
%matplotlib inline
from matplotlib import pyplot as plt
plt.rcParams['figure.figsize'] = (10, 8)
import seaborn as sns
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import collections
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

## Часть 1. Игрушечный набор данных "девушка в баре"

In [94]:
# Создание датафрейма с dummy variables
def create_df(dic, feature_list):
    out = pd.DataFrame(dic)
    out = pd.concat([out, pd.get_dummies(out[feature_list])], axis = 1)
    out.drop(feature_list, axis = 1, inplace = True)
    return out

# Некоторые значения признаков есть в тесте, но нет в трейне и наоборот
def intersect_features(train, test):
    common_feat = list( set(train.keys()) & set(test.keys()))
    return train[common_feat], test[common_feat]


In [95]:
features = ['Внешность', 'Алкоголь_в_напитке',
            'Уровень_красноречия', 'Потраченные_деньги']

In [96]:
df_train = {}
df_train['Внешность'] = ['приятная', 'приятная', 'приятная', 'отталкивающая',
                         'отталкивающая', 'отталкивающая', 'приятная'] 
df_train['Алкоголь_в_напитке'] = ['да', 'да', 'нет', 'нет', 'да', 'да', 'да']
df_train['Уровень_красноречия'] = ['высокий', 'низкий', 'средний', 'средний', 'низкий',
                                   'высокий', 'средний']
df_train['Потраченные_деньги'] = ['много', 'мало', 'много', 'мало', 'много',
                                  'много', 'много']
df_train['Поедет'] = LabelEncoder().fit_transform(['+', '-', '+', '-', '-', '+', '+'])

df_train = create_df(df_train, features)
df_train

,Поедет,Внешность_отталкивающая,Внешность_приятная,Алкоголь_в_напитке_да,Алкоголь_в_напитке_нет,Уровень_красноречия_высокий,Уровень_красноречия_низкий,Уровень_красноречия_средний,Потраченные_деньги_мало,Потраченные_деньги_много
0,0,0,1,1,0,1,0,0,0,1
1,1,0,1,1,0,0,1,0,1,0
2,0,0,1,0,1,0,0,1,0,1
3,1,1,0,0,1,0,0,1,1,0
4,1,1,0,1,0,0,1,0,0,1
5,0,1,0,1,0,1,0,0,0,1
6,0,0,1,1,0,0,0,1,0,1


In [97]:
df_test = {}
df_test['Внешность'] = ['приятная', 'приятная', 'отталкивающая'] 
df_test['Алкоголь_в_напитке'] = ['нет', 'да', 'да']
df_test['Уровень_красноречия'] = ['средний', 'высокий', 'средний']
df_test['Потраченные_деньги'] = ['много', 'мало', 'много']
df_test = create_df(df_test, features)
df_test

,Внешность_отталкивающая,Внешность_приятная,Алкоголь_в_напитке_да,Алкоголь_в_напитке_нет,Уровень_красноречия_высокий,Уровень_красноречия_средний,Потраченные_деньги_мало,Потраченные_деньги_много
0,0,1,0,1,0,1,0,1
1,0,1,1,0,1,0,1,0
2,1,0,1,0,0,1,0,1


In [98]:
# Некоторые значения признаков есть в тесте, но нет в трейне и наоборот
y=df_train["Поедет"]
df_train, df_test = intersect_features(train=df_train, test=df_test)
df_train

,Уровень_красноречия_высокий,Потраченные_деньги_много,Потраченные_деньги_мало,Алкоголь_в_напитке_нет,Внешность_отталкивающая,Уровень_красноречия_средний,Алкоголь_в_напитке_да,Внешность_приятная
0,1,1,0,0,0,0,1,1
1,0,0,1,0,0,0,1,1
2,0,1,0,1,0,1,0,1
3,0,0,1,1,1,1,0,0
4,0,1,0,0,1,0,1,0
5,1,1,0,0,1,0,1,0
6,0,1,0,0,0,1,1,1


In [99]:
df_test

,Уровень_красноречия_высокий,Потраченные_деньги_много,Потраченные_деньги_мало,Алкоголь_в_напитке_нет,Внешность_отталкивающая,Уровень_красноречия_средний,Алкоголь_в_напитке_да,Внешность_приятная
0,0,1,0,1,0,1,0,1
1,1,0,1,0,0,0,1,1
2,0,1,0,0,1,1,1,0


## Вопрос 1. Какова энтропия начальной системы (S0)? Под состояниями системы понимаем значения признака "Поедет" – 0 или 1 (то есть всего 2 состояния).

In [100]:
def ent(y):
    vals=list(set(y))
    t=[y.count(i)/len(y) for i in vals]
    t=sum([-i*np.log2(i) for i in t])
    return t

In [101]:
def information_gain (left,right):
    root=left+right
    ent_group=( ent(left)*len(left)/len(root) +
               ent(right)*len(right)/len(root) )
    return(ent(root)-ent_group)

In [102]:
print('The entropy of "Поехал" feature:{0:.3f}'.format(ent(list(y))))

The entropy of "Поехал" feature:0.985


## Вопрос 2. Рассмотрим разбиение обучающей выборки по признаку "Внешность_приятная". Какова энтропия S1 левой группы, тех, у кого внешность приятная, и правой группы – S2? Каков прирост информации при данном разбиении (IG)?

In [103]:
a=np.asarray(df_train["Внешность_приятная"],dtype=bool)
b=~a
a=list(y[a])
b=list(y[b])
print("s1:",a)
print("s2:",b)

s1: [0, 1, 0, 0]
s2: [1, 1, 0]


In [104]:
print("entropy of s1: {0:.3f}\nentropy of s2: {1:.3f}"
      .format(ent(a),ent(b)))
print("Information gain of splitting: {0:.3f}".format(information_gain(a,b)))

entropy of s1: 0.811
entropy of s2: 0.918
Information gain of splitting: 0.128


### Постройте с помощью sklearn дерево решений, обучив его на обучающей выборке. Глубину можно не ограничивать.

In [105]:
clf = DecisionTreeClassifier(random_state=0)
clf.fit(X=df_train,y=y)
print("predicting on the train set:")
clf.predict(df_train)==y

predicting on the train set:


0    True
1    True
2    True
3    True
4    True
5    True
6    True
Name: Поедет, dtype: bool

## Часть 2. Функции для расчета энтропии и прироста информации

In [106]:
balls = [1 for i in range(9)] + [0 for i in range(11)]

In [107]:
# две группы
balls_left  = [1 for i in range(8)] + [0 for i in range(5)] # 8 синих и 5 желтых
balls_right = [1 for i in range(1)] + [0 for i in range(6)] # 1 синий и 6 желтых

## Вопрос 3. Чему равна энтропия состояния, заданного списком balls_left?

In [108]:
print("entropy of balls_left: {0:.3f}".format(ent(balls_left)))

entropy of balls_left: 0.961


## Вопрос 4. Чему равна энтропия игральной кости с несмещенным центром тяжести?

In [109]:
print("entropy of an unbiased 6-side dice: {0:.3f}".format(ent([1,2,3,4,5,6])))

entropy of an unbiased 6-side dice: 2.585


## Вопрос 5. Каков прирост информации при разделении выборки на balls_left и balls_right?

In [110]:
print("Information gain of splitting into balls_left and balls_right groups: {0:.3f}"
      .format(information_gain(balls_left,balls_right)))

Information gain of splitting into balls_left and balls_right groups: 0.161


## Часть 3. Набор данных "Adult"

In [111]:
data_train = pd.read_csv('adult_train.csv', sep=',')
data_test = pd.read_csv('adult_test.csv', sep=',') 

In [112]:
data_train.shape

(32561, 15)

In [113]:
data_test=data_test.iloc[1:,:]

In [114]:
# необходимо убрать строки с неправильными метками в тестовой выборке
data_test = data_test[(data_test['Target'] == ' >50K.') 
                      | (data_test['Target']==' <=50K.')]

# перекодируем target в числовое поле
data_train.at[data_train['Target'] == ' <=50K', 'Target'] = 0
data_train.at[data_train['Target'] == ' >50K', 'Target'] = 1

data_test.at[data_test['Target'] == ' <=50K.', 'Target'] = 0
data_test.at[data_test['Target'] == ' >50K.', 'Target'] = 1

In [115]:
data_test.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Age,16281,73,35,461,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Workclass,15318,8,Private,11210,NaN,NaN,NaN,NaN,NaN,NaN,NaN
fnlwgt,16281,NaN,NaN,NaN,189436,105715,13492,116736,177831,238384,1.4904e+06
Education,16281,16,HS-grad,5283,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Education_Num,16281,NaN,NaN,NaN,10.0729,2.56755,1,9,10,12,16
Martial_Status,16281,7,Married-civ-spouse,7403,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Occupation,15315,14,Prof-specialty,2032,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Relationship,16281,6,Husband,6523,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Race,16281,5,White,13946,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Sex,16281,2,Male,10860,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [116]:
data_train['Target'].value_counts()

0    24720
1     7841
Name: Target, dtype: int64

In [117]:
data_test['Age'] = data_test['Age'].astype(int)

In [118]:
data_test['fnlwgt'] = data_test['fnlwgt'].astype(int)
data_test['Education_Num'] = data_test['Education_Num'].astype(int)
data_test['Capital_Gain'] = data_test['Capital_Gain'].astype(int)
data_test['Capital_Loss'] = data_test['Capital_Loss'].astype(int)
data_test['Hours_per_week'] = data_test['Hours_per_week'].astype(int)

In [119]:
# выделим в выборках категориальные и числовые поля

categorical_columns_train = [c for c in data_train.columns 
                             if data_train[c].dtype.name == 'object']
numerical_columns_train = [c for c in data_train.columns 
                           if data_train[c].dtype.name != 'object']

categorical_columns_test = [c for c in data_test.columns 
                            if data_test[c].dtype.name == 'object']
numerical_columns_test = [c for c in data_test.columns 
                          if data_test[c].dtype.name != 'object']

print('categorical_columns_test:', categorical_columns_test)
print('categorical_columns_train:', categorical_columns_train)
print('numerical_columns_test:', numerical_columns_test)
print('numerical_columns_train:', numerical_columns_train)

categorical_columns_test: ['Workclass', 'Education', 'Martial_Status', 'Occupation', 'Relationship', 'Race', 'Sex', 'Country']
categorical_columns_train: ['Workclass', 'Education', 'Martial_Status', 'Occupation', 'Relationship', 'Race', 'Sex', 'Country']
numerical_columns_test: ['Age', 'fnlwgt', 'Education_Num', 'Capital_Gain', 'Capital_Loss', 'Hours_per_week', 'Target']
numerical_columns_train: ['Age', 'fnlwgt', 'Education_Num', 'Capital_Gain', 'Capital_Loss', 'Hours_per_week', 'Target']


In [120]:
# заполним пропуски

for c in categorical_columns_train:
    data_train[c] = data_train[c].fillna(data_train[c].mode())
for c in categorical_columns_test:
    data_test[c] = data_test[c].fillna(data_train[c].mode())
    
for c in numerical_columns_train:
    data_train[c] = data_train[c].fillna(data_train[c].median())
for c in numerical_columns_test:
    data_test[c] = data_test[c].fillna(data_train[c].median())    

In [121]:
data_train = pd.concat([data_train, pd.get_dummies(data_train['Workclass'], 
                                                   prefix="Workclass"),
                      pd.get_dummies(data_train['Education'], prefix="Education"),
                      pd.get_dummies(data_train['Martial_Status'], prefix="Martial_Status"),
                      pd.get_dummies(data_train['Occupation'], prefix="Occupation"),
                      pd.get_dummies(data_train['Relationship'], prefix="Relationship"),
                      pd.get_dummies(data_train['Race'], prefix="Race"),
                      pd.get_dummies(data_train['Sex'], prefix="Sex"),
                      pd.get_dummies(data_train['Country'], prefix="Country")],
                     axis=1)

data_test = pd.concat([data_test, pd.get_dummies(data_test['Workclass'], prefix="Workclass"),
                      pd.get_dummies(data_test['Education'], prefix="Education"),
                      pd.get_dummies(data_test['Martial_Status'], prefix="Martial_Status"),
                      pd.get_dummies(data_test['Occupation'], prefix="Occupation"),
                      pd.get_dummies(data_test['Relationship'], prefix="Relationship"),
                      pd.get_dummies(data_test['Race'], prefix="Race"),
                      pd.get_dummies(data_test['Sex'], prefix="Sex"),
                      pd.get_dummies(data_test['Country'], prefix="Country")],
                     axis=1)

In [122]:
data_train.drop(['Workclass', 'Education', 'Martial_Status',
                 'Occupation', 'Relationship', 'Race', 'Sex', 'Country'],
                axis=1, inplace=True)
data_test.drop(['Workclass', 'Education', 'Martial_Status', 'Occupation', 
                'Relationship', 'Race', 'Sex', 'Country'],
               axis=1, inplace=True)

In [123]:
data_test.describe(include='all').T

,count,mean,std,min,25%,50%,75%,max
Age,16281.0,38.767459,13.849187,17.0,28.0,37.0,48.0,90.0
fnlwgt,16281.0,189435.677784,105714.907671,13492.0,116736.0,177831.0,238384.0,1490400.0
Education_Num,16281.0,10.072907,2.567545,1.0,9.0,10.0,12.0,16.0
Capital_Gain,16281.0,1081.905104,7583.935968,0.0,0.0,0.0,0.0,99999.0
Capital_Loss,16281.0,87.899269,403.105286,0.0,0.0,0.0,0.0,3770.0
Hours_per_week,16281.0,40.392236,12.479332,1.0,40.0,40.0,45.0,99.0
Target,16281.0,0.236226,0.424776,0.0,0.0,0.0,0.0,1.0
Workclass_ Federal-gov,16281.0,0.028991,0.167786,0.0,0.0,0.0,0.0,1.0
Workclass_ Local-gov,16281.0,0.064062,0.244872,0.0,0.0,0.0,0.0,1.0
Workclass_ Never-worked,16281.0,0.000184,0.013574,0.0,0.0,0.0,0.0,1.0


In [124]:
data_test['Country_ Holand-Netherlands'] = np.zeros([data_test.shape[0], 1])

In [125]:
data_train.head(2)

,Age,fnlwgt,Education_Num,Capital_Gain,Capital_Loss,Hours_per_week,Target,Workclass_ Federal-gov,Workclass_ Local-gov,Workclass_ Never-worked,...,Country_ Portugal,Country_ Puerto-Rico,Country_ Scotland,Country_ South,Country_ Taiwan,Country_ Thailand,Country_ Trinadad&Tobago,Country_ United-States,Country_ Vietnam,Country_ Yugoslavia
0,39,77516,13,2174,0,40,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,50,83311,13,0,0,13,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [126]:
data_test.head(2)

,Age,fnlwgt,Education_Num,Capital_Gain,Capital_Loss,Hours_per_week,Target,Workclass_ Federal-gov,Workclass_ Local-gov,Workclass_ Never-worked,...,Country_ Puerto-Rico,Country_ Scotland,Country_ South,Country_ Taiwan,Country_ Thailand,Country_ Trinadad&Tobago,Country_ United-States,Country_ Vietnam,Country_ Yugoslavia,Country_ Holand-Netherlands
1,25,226802,7,0,0,40,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0.0
2,38,89814,9,0,0,50,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0.0


## Вопрос 6. Какова доля правильных ответов дерева решений на тестовой выборке при максимальной глубине дерева = 3 и random_state = 17?

In [127]:
X_train=data_train.drop(['Target'], axis=1)
y_train = data_train['Target']

X_test=data_test.drop(['Target'], axis=1)
y_test = data_test['Target']

In [128]:
tree = DecisionTreeClassifier(random_state=17, max_depth=3)
tree.fit(X_train,y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=17,
            splitter='best')

In [129]:
y_train

0        0
1        0
2        0
3        0
4        0
5        0
6        0
7        1
8        1
9        1
10       1
11       1
12       0
13       0
14       1
15       0
16       0
17       0
18       0
19       1
20       1
21       0
22       0
23       0
24       0
25       1
26       0
27       1
28       0
29       0
        ..
32531    0
32532    1
32533    1
32534    0
32535    0
32536    1
32537    0
32538    1
32539    1
32540    0
32541    0
32542    0
32543    0
32544    0
32545    1
32546    0
32547    0
32548    0
32549    0
32550    0
32551    0
32552    0
32553    0
32554    1
32555    0
32556    0
32557    1
32558    0
32559    0
32560    1
Name: Target, Length: 32561, dtype: int64

In [130]:
tree_predictions=tree.predict(X_test)
acc=accuracy_score(y_test,tree_predictions)
print("Fraction of correct predictions on test set: {0:.3f}".format(acc))

Fraction of correct predictions on test set: 0.845


## Вопрос 7. Какова доля правильных ответов дерева решений на тестовой выборке при максимальной глубине дерева = 9 и random_state = 17?

In [131]:
tree = DecisionTreeClassifier(random_state=17, max_depth=9)
tree.fit(X_train,y_train)
tree_predictions=tree.predict(X_test)
acc=accuracy_score(y_test,tree_predictions)
print("Fraction of correct predictions on test set: {0:.3f}".format(acc))

Fraction of correct predictions on test set: 0.847


## Случайный лес без настройки параметров

Обучите на имеющейся выборке случайный лес (RandomForestClassifier), число деревьев сделайте равным ста, а random_state = 17. Сделайте с помощью полученной модели прогноз для тестовой выборки.

In [350]:
clf = RandomForestClassifier(n_estimators=100, random_state=17)
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)
acc=accuracy_score(y_test,y_pred)
print("Fraction of correct predictions on test set: {0:.3f}".format(acc))

Fraction of correct predictions on test set: 0.858


## Случайный лес с настройкой параметров (опционально)

Обучите на имеющейся выборке случайный лес (RandomForestClassifier). Максимальную глубину и максимальное число признаков для каждого дерева настройте с помощью GridSearchCV.

In [ ]:
forest_params = {'max_depth': range(10, 21),
                'max_features': range(5, 105, 10)}

locally_best_forest = GridSearchCV(clf,forest_params,cv=3,scoring="accuracy")
locally_best_forest.fit(X=X_train,y=y_train) 

In [309]:
print("Best params:", locally_best_forest.best_params_)
print("Best cross validaton score {0:.3f}".format(locally_best_forest.best_score_))

Best params: {'max_depth': 14, 'max_features': 45}
Best cross validaton score 0.865


Сделайте с помощью полученной модели прогноз для тестовой выборки.

In [308]:
tuned_forest_predictions = locally_best_forest.predict(X_test)
acc=accuracy_score(y_test,tuned_forest_predictions)
print("Fraction of correct predictions on test set: {0:.3f}".format(acc))

Fraction of correct predictions on test set: 0.866
